In [107]:
from sqlalchemy import create_engine, Column, Integer, String, Text, Float, ForeignKey, DECIMAL, TIMESTAMP, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import declarative_base, relationship, sessionmaker,Session
from datetime import datetime
import time
from dotenv import load_dotenv
import os
from sqlalchemy import MetaData
from concurrent.futures import ThreadPoolExecutor

In [ ]:
load_dotenv()
database_url = os.getenv("DATABASE_URL")

In [108]:
engine = create_engine(
    database_url,
    echo=False,
)

Base = declarative_base()

metadata = MetaData()
with engine.connect() as connection:
    connection.execute(text("CREATE SCHEMA IF NOT EXISTS urbanpulse"))
    connection.commit()

2025-03-03 13:22:11,804 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-03-03 13:22:11,806 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-03 13:22:11,987 INFO sqlalchemy.engine.Engine select current_schema()
2025-03-03 13:22:11,989 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-03 13:22:12,171 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-03-03 13:22:12,173 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-03 13:22:12,371 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-03 13:22:12,373 INFO sqlalchemy.engine.Engine CREATE SCHEMA IF NOT EXISTS urbanpulse
2025-03-03 13:22:12,374 INFO sqlalchemy.engine.Engine [generated in 0.00238s] {}
2025-03-03 13:22:12,554 INFO sqlalchemy.engine.Engine COMMIT


In [109]:
# Citizen Table
class Citizen(Base):
    __tablename__ = "citizen"
    __table_args__ = {"schema": "urbanpulse"}
    citizen_id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String(50))
    last_name = Column(String(50))
    full_name = Column(String(101))
    sex = Column(String(10))
    email = Column(String(255), unique=True)
    contact_number = Column(String(20))
    password = Column(String(255))
    address = Column(Text)
    created_at = Column(TIMESTAMP, default=datetime.utcnow)

    issues = relationship("Issue", back_populates="citizen")
    votes = relationship("Vote", back_populates="citizen")


In [110]:
# Department Table
class Department(Base):
    __tablename__ = "department"
    __table_args__ = {"schema": "urbanpulse"}
    department_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(100), unique=True)
    budget_points = Column(DECIMAL(12, 2))
    created_at = Column(TIMESTAMP, default=datetime.utcnow)

In [111]:
# Official Table
class Official(Base):
    __tablename__ = "official"
    __table_args__ = {"schema": "urbanpulse"}
    official_id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String(50))
    last_name = Column(String(50))
    full_name = Column(String(101))
    sex = Column(String(10))
    email = Column(String(255), unique=True)
    contact_number = Column(String(20))
    password = Column(String(255))
    address = Column(Text)
    # Fully qualify the referenced table name in the ForeignKey string:
    department_id = Column(Integer, ForeignKey("urbanpulse.department.department_id"))
    created_at = Column(TIMESTAMP, default=datetime.utcnow)

In [112]:
# Issue Table
class Issue(Base):
    __tablename__ = "issue"
    __table_args__ = {"schema": "urbanpulse"}
    issue_id = Column(Integer, primary_key=True, autoincrement=True)
    citizen_id = Column(Integer, ForeignKey("urbanpulse.citizen.citizen_id"))
    description = Column(Text)
    category = Column(String(50))
    priority_level = Column(Integer)
    latitude = Column(Float)
    longitude = Column(Float)
    status = Column(String(20))
    created_at = Column(TIMESTAMP, default=datetime.utcnow)

    citizen = relationship("Citizen", back_populates="issues")
    votes = relationship("Vote", back_populates="issue")

In [113]:
# Photo Table
class Photo(Base):
    __tablename__ = "photo"
    __table_args__ = {"schema": "urbanpulse"}
    photo_id = Column(Integer, primary_key=True, autoincrement=True)
    issue_id = Column(Integer, ForeignKey("urbanpulse.issue.issue_id"))
    photo_url = Column(String(512))
    created_at = Column(TIMESTAMP, default=datetime.utcnow)

In [114]:
# Vote Table
class Vote(Base):
    __tablename__ = "vote"
    __table_args__ = {"schema": "urbanpulse"}
    vote_id = Column(Integer, primary_key=True, autoincrement=True)
    citizen_id = Column(Integer, ForeignKey("urbanpulse.citizen.citizen_id"))
    issue_id = Column(Integer, ForeignKey("urbanpulse.issue.issue_id"))
    priority_vote = Column(Integer)
    created_at = Column(TIMESTAMP, default=datetime.utcnow)

    citizen = relationship("Citizen", back_populates="votes")
    issue = relationship("Issue", back_populates="votes")

In [115]:
# Transaction Table
class Transaction(Base):
    __tablename__ = "transaction"
    __table_args__ = {"schema": "urbanpulse"}
    transaction_id = Column(Integer, primary_key=True, autoincrement=True)
    issue_id = Column(Integer, ForeignKey("urbanpulse.issue.issue_id"))
    official_id = Column(Integer, ForeignKey("urbanpulse.official.official_id"))
    budget_spent = Column(DECIMAL(10, 2))
    status = Column(String(20))
    created_at = Column(TIMESTAMP, default=datetime.utcnow)

In [116]:
# Database setup
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

2025-03-03 13:22:12,697 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-03 13:22:12,698 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_namespace.nspname = %(nspname_1)s
2025-03-03 13:22:12,698 INFO sqlalchemy.engine.Engine [generated in 0.00033s] {'table_name': 'citizen', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'urbanpulse'}
2025-03-03 13:22:12,879 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind

In [117]:
# Sample data
# Departments
dept1 = Department(name="Infrastructure", budget_points=50000)
dept2 = Department(name="Health", budget_points=75000)

# Officials
official1 = Official(first_name="John", last_name="Doe", full_name="John Doe",
                     sex="Male", email="johndoe@email.com", contact_number="1234567890",
                     password="hashedpassword", address="City Hall, NY", department_id=1)

official2 = Official(first_name="Jane", last_name="Smith", full_name="Jane Smith",
                     sex="Female", email="janesmith@email.com", contact_number="0987654321",
                     password="hashedpassword", address="Health Office, NY", department_id=2)

# Citizens
citizen1 = Citizen(first_name="Alice", last_name="Brown", full_name="Alice Brown",
                   sex="Female", email="alice@email.com", contact_number="1112223333",
                   password="hashedpassword", address="123 Main St, NY")

citizen2 = Citizen(first_name="Bob", last_name="Miller", full_name="Bob Miller",
                   sex="Male", email="bob@email.com", contact_number="4445556666",
                   password="hashedpassword", address="456 Elm St, NY")

# Issues
issue1 = Issue(citizen_id=1, description="Broken streetlights",
               category="Infrastructure", priority_level=2, latitude=40.7128, longitude=-74.0060,
               status="Pending")

issue2 = Issue(citizen_id=2, description="Public park needs maintenance",
               category="Environment", priority_level=3, latitude=40.7138, longitude=-74.0070,
               status="Pending")

# Add parent objects first
session.add_all([
    dept1, dept2,
    official1, official2,
    citizen1, citizen2,
    issue1, issue2
])
session.flush()

# Photos
photo1 = Photo(issue_id=issue1.issue_id, photo_url="https://example.com/photo1.jpg")
photo2 = Photo(issue_id=issue2.issue_id, photo_url="https://example.com/photo2.jpg")

# Votes
vote1 = Vote(citizen_id=1, issue_id=issue1.issue_id, priority_vote=5)
vote2 = Vote(citizen_id=2, issue_id=issue2.issue_id, priority_vote=4)

# Transactions
transaction1 = Transaction(issue_id=issue1.issue_id, official_id=1, budget_spent=1000, status="Approved")
transaction2 = Transaction(issue_id=issue2.issue_id, official_id=2, budget_spent=2000, status="Pending")

# Add dependent objects
session.add_all([
    photo1, photo2,
    vote1, vote2,
    transaction1, transaction2
])
session.commit()

2025-03-03 13:22:14,247 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-03 13:22:14,249 INFO sqlalchemy.engine.Engine INSERT INTO urbanpulse.citizen (first_name, last_name, full_name, sex, email, contact_number, password, address, created_at) SELECT p0::VARCHAR, p1::VARCHAR, p2::VARCHAR, p3::VARCHAR, p4::VARCHAR, p5::VARCHAR, p6::VARCHAR, p7::TEXT, p8::TIMESTAMP WIT ... 418 characters truncated ...  sen_counter RETURNING urbanpulse.citizen.citizen_id, urbanpulse.citizen.citizen_id AS citizen_id__1
2025-03-03 13:22:14,250 INFO sqlalchemy.engine.Engine [generated in 0.00013s (insertmanyvalues) 1/1 (ordered)] {'last_name__0': 'Brown', 'first_name__0': 'Alice', 'full_name__0': 'Alice Brown', 'address__0': '123 Main St, NY', 'password__0': 'hashedpassword', 'email__0': 'alice@email.com', 'created_at__0': datetime.datetime(2025, 3, 3, 6, 22, 14, 249508), 'contact_number__0': '1112223333', 'sex__0': 'Female', 'last_name__1': 'Miller', 'first_name__1': 'Bob', 'full_name__1': 'Bob Miller

In [118]:
def create_new_citizen(first_name, last_name, sex, email, contact_number, password, address):
    session = Session()
    citizen = Citizen(
        first_name=first_name,
        last_name=last_name,
        full_name=f"{first_name} {last_name}",
        sex=sex,
        email=email,
        contact_number=contact_number,
        password=password,
        address=address
    )
    session.add(citizen)
    session.commit()
    session.close()
    print(f"Created citizen: {citizen.full_name}")
    return citizen

In [119]:
def report_issue(citizen_id, description, category, priority_level, latitude, longitude, status="Pending"):
    session = Session()
    try:
        issue = Issue(
            citizen_id=citizen_id,
            description=description,
            category=category,
            priority_level=priority_level,
            latitude=latitude,
            longitude=longitude,
            status=status
        )
        session.add(issue)
        session.commit()
        issue_id = issue.issue_id  # Extract ID while session is active
        print(f"Issue reported: {description}")
        return issue_id  # Return the ID instead of the Issue object
    finally:
        session.close()

In [120]:
def cast_vote(citizen_id, issue_id, priority_vote):
    session = Session()
    vote = Vote(
        citizen_id=citizen_id,
        issue_id=issue_id,
        priority_vote=priority_vote
    )
    session.add(vote)
    session.commit()
    session.close()
    print(f"Vote cast on issue {issue_id} by citizen {citizen_id}")
    return vote

In [121]:
def process_issue_transaction(issue_id, official_id, budget_spent):
    """
    This operation processes a transaction for an issue.
    It locks the corresponding department row to ensure that the deduction from budget is atomic.
    """
    session = Session()
    try:
        official = session.query(Official).filter(Official.official_id == official_id).one()
        # Lock the department row for update to prevent race conditions
        department = session.query(Department)\
            .with_for_update()\
            .filter(Department.department_id == official.department_id)\
            .one()
        if department.budget_points < budget_spent:
            session.rollback()
            raise Exception("Not enough budget in the department.")
        # Deduct the budget points
        department.budget_points -= budget_spent
        # Create a transaction record
        transaction_record = Transaction(
            issue_id=issue_id,
            official_id=official_id,
            budget_spent=budget_spent,
            status='Approved'
        )
        session.add(transaction_record)
        session.commit()
        print(f"Processed transaction for issue {issue_id} by official {official_id}")
        return transaction_record
    except Exception as e:
        session.rollback()
        print(f"Transaction failed: {e}")
    finally:
        session.close()
    return None


In [122]:
# --------------------------
# Performance Test Functions
# --------------------------

def performance_test_report_issue(num_iterations=10000):
    """Test performance of reporting issues."""
    start = time.time()
    session = Session()
    citizen = session.query(Citizen).first()
    if not citizen:
        citizen = create_new_citizen("Test", "User", "Other", "testuser@example.com", "0000000000", "password", "Test Address")
    session.close()
    for i in range(num_iterations):
        report_issue(citizen.citizen_id, f"Issue {i}", "Infrastructure", 3, 12.34, 56.78)
    end = time.time()
    print(f"Performance Test - Reporting {num_iterations} issues took {end - start:.2f} seconds.")

In [123]:
def performance_test_cast_vote(num_iterations=10000):
    start = time.time()
    session = Session()
    citizen = session.query(Citizen).first()
    issue = session.query(Issue).first()
    if not citizen:
        citizen = create_new_citizen("Test", "User2", "Other", "testuser2@example.com", "0000000000", "password", "Test Address 2")
    if not issue:
        issue_id = report_issue(citizen.citizen_id, "Test issue for votes", "Infrastructure", 3, 12.34, 56.78)
    else:
        issue_id = issue.issue_id  # Use existing issue's ID if available
    session.close()
    for i in range(num_iterations):
        cast_vote(citizen.citizen_id, issue_id, 3)
    end = time.time()
    print(f"Performance Test - Casting {num_iterations} votes took {end - start:.2f} seconds.")

In [124]:
# --------------------------
# Isolation Test Function
# --------------------------

def isolation_test_process_issue_transaction():
    session = Session()
    try:
        # Query or create department
        department = session.query(Department).filter_by(name='Isolation Test Dept').first()
        if not department:
            department = Department(name='Isolation Test Dept', budget_points=1000.00)
            session.add(department)
            session.commit()

        # Query or create official
        official = session.query(Official).filter_by(email='isol@official.com').first()
        if not official:
            official = Official(
                first_name='Iso', last_name='Test', full_name='Iso Test', sex='Other',
                email='isol@official.com', contact_number='111111', password='password',
                address='Test Address', department_id=department.department_id
            )
            session.add(official)
            session.commit()

        # Query or create citizen
        citizen = session.query(Citizen).first()
        if not citizen:
            citizen = create_new_citizen("Iso", "Citizen", "Other", "iso@citizen.com", "2222222222", "password", "Test Address")

        # Report the issue and get the issue_id
        issue_id = report_issue(citizen.citizen_id, "Isolation test issue", "Infrastructure", 3, 12.34, 56.78)
        official_id = official.official_id  # Extract official ID

        # Define the wrapper function using only IDs
        def process_transaction_wrapper(budget_spent):
            return process_issue_transaction(issue_id, official_id, budget_spent)

        # Execute concurrent transactions
        with ThreadPoolExecutor(max_workers=2) as executor:
            future1 = executor.submit(process_transaction_wrapper, 700)
            future2 = executor.submit(process_transaction_wrapper, 700)
            result1 = future1.result()
            result2 = future2.result()

        # Query the updated department budget
        updated_department = session.query(Department).filter(Department.department_id == department.department_id).one()
        print(f"Isolation Test - After concurrent transactions, department budget: {updated_department.budget_points}")
        print("Isolation test completed. Only one transaction should succeed if budget is insufficient.")

    finally:
        session.close()

In [ ]:
# Main block to run tests
if __name__ == '__main__':
    # Optional: Reset sequence or clear tables
    with engine.connect() as connection:
        connection.execute(text("SELECT setval('urbanpulse.issue_issue_id_seq', (SELECT COALESCE(MAX(issue_id), 0) + 1 FROM urbanpulse.issue))"))
        connection.commit()

    # Run performance tests
    performance_test_report_issue(num_iterations=1000)
    performance_test_cast_vote(num_iterations=1000)

    # Run isolation test
    isolation_test_process_issue_transaction()

2025-03-03 13:25:10,256 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-03 13:25:10,258 INFO sqlalchemy.engine.Engine SELECT setval('urbanpulse.issue_issue_id_seq', (SELECT COALESCE(MAX(issue_id), 0) + 1 FROM urbanpulse.issue))
2025-03-03 13:25:10,259 INFO sqlalchemy.engine.Engine [cached since 175s ago] {}
2025-03-03 13:25:10,442 INFO sqlalchemy.engine.Engine COMMIT
2025-03-03 13:25:10,536 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-03 13:25:10,538 INFO sqlalchemy.engine.Engine SELECT urbanpulse.citizen.citizen_id AS urbanpulse_citizen_citizen_id, urbanpulse.citizen.first_name AS urbanpulse_citizen_first_name, urbanpulse.citizen.last_name AS urbanpulse_citizen_last_name, urbanpulse.citizen.full_name AS urbanpulse_citizen_full_name, urbanpulse.citizen.sex AS urbanpulse_citizen_sex, urbanpulse.citizen.email AS urbanpulse_citizen_email, urbanpulse.citizen.contact_number AS urbanpulse_citizen_contact_number, urbanpulse.citizen.password AS urbanpulse_citizen_password, urb